In [1]:
!sudo apt install linux-tools-generic bpftool zstd

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'linux-tools-common' instead of 'bpftool'
linux-tools-generic is already the newest version (6.2.0.39.39).
linux-tools-common is already the newest version (6.2.0-39.40).
zstd is already the newest version (1.5.4+dfsg2-4).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


In [2]:
import os


def system(cmd):
    print(f"Running: `{cmd}`")
    os.system(cmd)

# Getting BTF Files for Ubuntu 20.04

In [6]:
from pathlib import Path

data_path = Path("data/20.04-x86")
tmp_path = data_path / 'tmp'
tmp_path.mkdir(parents=True, exist_ok=True)

In [8]:
import urllib.request
import gzip

def download_package_index():
    url = "http://security.ubuntu.com/ubuntu/dists/focal-security/main/binary-amd64/Packages.gz"
    gz_path = tmp_path / "Packages.gz"

    if not gz_path.exists():
        print(f"Downloading {url} to {gz_path}")
        urllib.request.urlretrieve(url, gz_path)
    else:
        print(f"Using {gz_path}")

    package_path = gz_path.with_suffix("")
    if not package_path.with_suffix("").exists():
        print(f"Unzipping {gz_path} to {package_path}")
        with gzip.open(gz_path, 'rb') as f_in:
            with open(package_path, 'wb') as f_out:
                f_out.write(f_in.read())
    else:
        print(f"Using {package_path}")

    return package_path


package_path = download_package_index()
package_path

Using data/20.04-x86/tmp/Packages.gz
Using data/20.04-x86/tmp/Packages


PosixPath('data/20.04-x86/tmp/Packages')

In [10]:
def parse_package_index(package_path):
    result = {}
    with open(package_path) as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            key, val = line.split(": ", 1)
            if key == "Package":
                package = val
            elif key == "Filename":
                result[package] = val
    return result


package_index = parse_package_index(package_path)

package_index

{'accountsservice': 'pool/main/a/accountsservice/accountsservice_0.6.55-0ubuntu12~20.04.6_amd64.deb',
 'adcli': 'pool/main/a/adcli/adcli_0.9.0-1ubuntu0.20.04.1_amd64.deb',
 'adsys': 'pool/main/a/adsys/adsys_0.9.2~20.04.1_amd64.deb',
 'adsys-windows': 'pool/main/a/adsys/adsys-windows_0.9.2~20.04.1_amd64.deb',
 'advancecomp': 'pool/main/a/advancecomp/advancecomp_2.1-2.1ubuntu0.20.04.1_amd64.deb',
 'aide': 'pool/main/a/aide/aide_0.16.1-1ubuntu0.1_amd64.deb',
 'aide-common': 'pool/main/a/aide/aide-common_0.16.1-1ubuntu0.1_all.deb',
 'amd64-microcode': 'pool/main/a/amd64-microcode/amd64-microcode_3.20191218.1ubuntu1.2_amd64.deb',
 'apache2': 'pool/main/a/apache2/apache2_2.4.41-4ubuntu3.15_amd64.deb',
 'apache2-bin': 'pool/main/a/apache2/apache2-bin_2.4.41-4ubuntu3.15_amd64.deb',
 'apache2-data': 'pool/main/a/apache2/apache2-data_2.4.41-4ubuntu3.15_all.deb',
 'apache2-dev': 'pool/main/a/apache2/apache2-dev_2.4.41-4ubuntu3.15_amd64.deb',
 'apache2-doc': 'pool/main/a/apache2/apache2-doc_2.4.41

In [12]:
import re
from collections import defaultdict

def filter_linux_images(package_index):
    result = defaultdict(dict)
    for package, path in package_index.items():
        groups = re.match(
            r"linux-image-(\d+\.\d+\.\d+)-(\d+)-generic", package)
        if groups is None:
            continue

        version, build = groups.groups()
        result[version][int(build)] = package, path

    result = {k: v[max(v.keys())] for k, v in result.items()}
    return result


linux_versions = filter_linux_images(package_index)

linux_versions

{'5.11.0': ('linux-image-5.11.0-46-generic',
  'pool/main/l/linux-signed-hwe-5.11/linux-image-5.11.0-46-generic_5.11.0-46.51~20.04.1_amd64.deb'),
 '5.13.0': ('linux-image-5.13.0-52-generic',
  'pool/main/l/linux-signed-hwe-5.13/linux-image-5.13.0-52-generic_5.13.0-52.59~20.04.1_amd64.deb'),
 '5.15.0': ('linux-image-5.15.0-92-generic',
  'pool/main/l/linux-signed-hwe-5.15/linux-image-5.15.0-92-generic_5.15.0-92.102~20.04.1_amd64.deb'),
 '5.4.0': ('linux-image-5.4.0-170-generic',
  'pool/main/l/linux-signed/linux-image-5.4.0-170-generic_5.4.0-170.188_amd64.deb'),
 '5.8.0': ('linux-image-5.8.0-63-generic',
  'pool/main/l/linux-signed-hwe-5.8/linux-image-5.8.0-63-generic_5.8.0-63.71~20.04.1_amd64.deb')}

In [13]:
def download_deb_files(linux_versions):
    results = {}
    for version, (package, path) in linux_versions.items():
        if version in ['5.4.0', '5.8.0']:  # we already have these
            continue

        url = f"http://security.ubuntu.com/ubuntu/{path}"
        file_path = tmp_path / path.split("/")[-1]
        if not file_path.exists():
            print(f"Downloading {url} to {file_path}")
            urllib.request.urlretrieve(url, file_path)
        else:
            print(f"Using {file_path}")

        key = package.removeprefix("linux-image-")
        results[key] = file_path

    return results


deb_paths = download_deb_files(linux_versions)

deb_paths

Using data/20.04-x86/tmp/linux-image-5.11.0-46-generic_5.11.0-46.51~20.04.1_amd64.deb
Using data/20.04-x86/tmp/linux-image-5.13.0-52-generic_5.13.0-52.59~20.04.1_amd64.deb
Using data/20.04-x86/tmp/linux-image-5.15.0-92-generic_5.15.0-92.102~20.04.1_amd64.deb


{'5.11.0-46-generic': PosixPath('data/20.04-x86/tmp/linux-image-5.11.0-46-generic_5.11.0-46.51~20.04.1_amd64.deb'),
 '5.13.0-52-generic': PosixPath('data/20.04-x86/tmp/linux-image-5.13.0-52-generic_5.13.0-52.59~20.04.1_amd64.deb'),
 '5.15.0-92-generic': PosixPath('data/20.04-x86/tmp/linux-image-5.15.0-92-generic_5.15.0-92.102~20.04.1_amd64.deb')}

In [14]:
def extract_vmlinuz_files(deb_paths):
    results = {}
    for name, deb_path in deb_paths.items():
        vmlinuz_path = tmp_path / "boot" / f"vmlinuz-{name}"
        if not vmlinuz_path.exists():
            print(f"Extracting {deb_path} to {vmlinuz_path}")
            system(f"dpkg-deb -x {deb_path} {tmp_path}")
        else:
            print(f"Using {vmlinuz_path}")
        results[name] = vmlinuz_path
    return results


vmlinuz_paths = extract_vmlinuz_files(deb_paths)

vmlinuz_paths

Using data/20.04-x86/tmp/boot/vmlinuz-5.11.0-46-generic
Using data/20.04-x86/tmp/boot/vmlinuz-5.13.0-52-generic
Using data/20.04-x86/tmp/boot/vmlinuz-5.15.0-92-generic


{'5.11.0-46-generic': PosixPath('data/20.04-x86/tmp/boot/vmlinuz-5.11.0-46-generic'),
 '5.13.0-52-generic': PosixPath('data/20.04-x86/tmp/boot/vmlinuz-5.13.0-52-generic'),
 '5.15.0-92-generic': PosixPath('data/20.04-x86/tmp/boot/vmlinuz-5.15.0-92-generic')}

In [15]:
def download_extract_vmlinux():
    result = tmp_path / Path("extract-vmlinux")
    if not result.exists():
        print(f"Downloading {result}")
        urllib.request.urlretrieve(
            "https://raw.githubusercontent.com/torvalds/linux/master/scripts/extract-vmlinux", result)
        system(f"chmod +x {result}")
    else:
        print(f"Using {result}")
    return result


extract_vmlinux = download_extract_vmlinux()

extract_vmlinux

Using data/20.04-x86/tmp/extract-vmlinux


PosixPath('data/20.04-x86/tmp/extract-vmlinux')

In [16]:
def extract_vmlinux_files(vmlinuz_paths):
    results = {}
    for name, vmlinuz_path in vmlinuz_paths.items():
        vmlinux_path = tmp_path / f"vmlinux-{name}"
        if not vmlinux_path.exists():
            print(f"Extracting {vmlinuz_path} to {vmlinux_path}")
            system(f"{extract_vmlinux} {vmlinuz_path} > {vmlinux_path}")
        else:
            print(f"Using {vmlinux_path}")
        results[name] = vmlinux_path
    return results


vmlinux_paths = extract_vmlinux_files(vmlinuz_paths)

vmlinux_paths

Using data/20.04-x86/tmp/vmlinux-5.11.0-46-generic
Using data/20.04-x86/tmp/vmlinux-5.13.0-52-generic
Using data/20.04-x86/tmp/vmlinux-5.15.0-92-generic


{'5.11.0-46-generic': PosixPath('data/20.04-x86/tmp/vmlinux-5.11.0-46-generic'),
 '5.13.0-52-generic': PosixPath('data/20.04-x86/tmp/vmlinux-5.13.0-52-generic'),
 '5.15.0-92-generic': PosixPath('data/20.04-x86/tmp/vmlinux-5.15.0-92-generic')}

In [18]:
def extract_btf_files(vmlinux_paths):
    results = {}
    for name, vmlinux_path in vmlinux_paths.items():
        btf_path = data_path / f"{name}.btf"
        if not btf_path.exists():
            print(f"Extracting {vmlinux_path} to {btf_path}")
            system(
                f"objcopy -I elf64-little {vmlinux_path} --dump-section .BTF={btf_path}")
            # we use objcopy instead of pahole because pahole sometimes fails with
            # "btf_encoder__new: cannot get ELF header", and pahole seems does more
            # processing than we need
            # system(f"pahole --btf_encode_detached {btf_path} {vmlinux_path}")
        else:
            print(f"Using {btf_path}")
        results[vmlinux_path.name] = btf_path

    return results


btf_paths = extract_btf_files(vmlinux_paths)

btf_paths

Using data/20.04-x86/5.11.0-46-generic.btf
Using data/20.04-x86/5.13.0-52-generic.btf
Using data/20.04-x86/5.15.0-92-generic.btf


{'vmlinux-5.11.0-46-generic': PosixPath('data/20.04-x86/5.11.0-46-generic.btf'),
 'vmlinux-5.13.0-52-generic': PosixPath('data/20.04-x86/5.13.0-52-generic.btf'),
 'vmlinux-5.15.0-92-generic': PosixPath('data/20.04-x86/5.15.0-92-generic.btf')}

# Generate JSON Files

In [43]:
from pathlib import Path

def get_linux_tools_path():
    parent = Path("/usr/lib/linux-tools")
    versions = [x for x in parent.iterdir() if x.is_dir()]
    if len(versions) == 0:
        raise Exception("No linux-tools found")
    versions.sort()
    return parent / versions[-1]


def get_bpftool_path():
    path = get_linux_tools_path() / "bpftool"
    if not path.exists():
        raise Exception("bpftool not found")
    return path


bpftool_path = get_bpftool_path()
bpftool_path

PosixPath('/usr/lib/linux-tools/6.2.0-39-generic/bpftool')

In [ ]:
def dump_btf(parent_path):
    for file in parent_path.glob("*.btf"):
        for ext, cmd in [(".h", "format c"), (".txt", "format raw"), (".json", "--json")]:
            result = file.with_suffix(ext)
            if not result.exists():
                system(f"{bpftool_path} btf dump file {file} {cmd} > {result}")
            else:
                print(f"{result} already exists")

for path in Path("data").glob("*"):
    if path.is_dir():
        dump_btf(path)

# Diffing JSON Files

In [2]:
from enum import Enum


class Kind(str, Enum):
    INT = "INT"
    PTR = "PTR"
    ARRAY = "ARRAY"
    STRUCT = "STRUCT"
    UNION = "UNION"
    ENUM = "ENUM"
    FWD = "FWD"
    TYPEDEF = "TYPEDEF"
    VOLATILE = "VOLATILE"
    CONST = "CONST"
    RESTRICT = "RESTRICT"
    FUNC = "FUNC"
    FUNC_PROTO = "FUNC_PROTO"
    VAR = "VAR"
    DATASEC = "DATASEC"
    FLOAT = "FLOAT"
    DECL_TAG = "DECL_TAG"
    TYPE_TAG = "TYPE_TAG"
    ENUM64 = "ENUM64"

In [4]:
from functools import cache
from pathlib import Path
import json


class BTF:
    def __init__(self, path):
        self.path = Path(path)
        with open(path) as f:
            self._raw_data = json.load(f)['types']

    def __getitem__(self, id):
        if id == 0:
            return {'id': 0, 'name': 'void', 'kind': 'VOID'}
        e = self._raw_data[id - 1]
        assert e['id'] == id
        return e

    def __len__(self):
        return len(self._raw_data)

    def __iter__(self):
        return iter(self._raw_data)

    @property
    def short_name(self):
        linux_version = self.path.name.split("-")[0]
        assert linux_version.endswith(".0")
        return linux_version[:-2]

    def print(self):
        from collections import defaultdict

        print(f"File: {self.path}")

        print("Sample:")
        kinds = defaultdict(int)
        for e in self:
            if e['kind'] not in kinds:
                print(f"\t{e['id']:6} ({e['kind']:10}): {e}")
                print(f"\t{'':18}-> {self.normalize(e['id'])}")
            kinds[e['kind']] += 1

        kinds = sorted(kinds.items(), key=lambda x: x[1], reverse=True)
        print(f"Kinds:")
        print(f"\t{dict(kinds)}")

        print()

    def filter_on_kind(self, kind):
        return {
            e['name']: self.normalize(e['id']) for e in self
            if e['kind'] == kind and e['name'] != '(anon)'
        }

    def get(self, kind, name):
        for elem in self:
            if elem['kind'] == kind and elem['name'] == name:
                return self.normalize(elem['id'])

    RECURSE_KINDS = {Kind.CONST, Kind.VOLATILE, Kind.RESTRICT,
                     Kind.PTR, Kind.FUNC, Kind.FUNC_PROTO, Kind.ARRAY}

    # @cache
    def normalize(self, type_id, recurse=True):
        elem = self[type_id].copy()

        # Recurse into types for certain kinds
        recurse = recurse or elem['kind'] in self.RECURSE_KINDS

        # Remove redundant fields
        del elem['id']

        kind = elem['kind']
        if kind == Kind.INT:
            assert elem['bits_offset'] == 0
            del elem['bits_offset']
            del elem['encoding']
            del elem['nr_bits']
            del elem['size']
        elif kind == Kind.ARRAY:
            del elem['index_type_id']
        elif kind == Kind.ENUM:
            if not recurse:
                assert elem['vlen'] == len(elem['values'])
                del elem['values']
                del elem['vlen']
                del elem['encoding']
        elif kind == Kind.FUNC:
            assert elem['linkage'] == 'static'
            del elem['linkage']
        elif kind in (Kind.PTR, Kind.FUNC_PROTO):
            assert elem['name'] == '(anon)'
            del elem['name']

        # Normalize types
        for type in ['type', 'ret_type']:
            type_id = f"{type}_id"

            if type_id not in elem:
                continue

            if recurse:
                elem[type] = self.normalize(elem[type_id], recurse=False)
            del elem[type_id]

        for list_key in ['params', 'members']:
            if list_key not in elem:
                continue

            assert len(elem[list_key]) == elem['vlen']
            del elem['vlen']

            if recurse:
                elem[list_key] = [
                    {
                        **{k: v for k, v in item.items() if k != 'type_id'},
                        'type': self.normalize(item['type_id'], recurse=False)
                    }
                    for item in elem[list_key]
                ]
            else:
                del elem[list_key]
                if list_key == 'members':
                    del elem['size']

        return elem


d1 = BTF("data/20.04-x86/5.13.0-52-generic.json")
d2 = BTF("data/20.04-x86/5.15.0-92-generic.json")

d1.get(Kind.UNION, "intel_x86_pebs_dse")

# d1.print()

# d1.get_by_kind_name(Kind.STRUCT, "task_struct")
# d1.get_by_kind_name(Kind.FUNC, "vfs_read")

{'kind': 'UNION',
 'name': 'intel_x86_pebs_dse',
 'size': 8,
 'members': [{'name': 'val',
   'bits_offset': 0,
   'type': {'kind': 'TYPEDEF', 'name': 'u64'}},
  {'name': '(anon)',
   'bits_offset': 0,
   'type': {'kind': 'STRUCT', 'name': '(anon)'}},
  {'name': '(anon)',
   'bits_offset': 0,
   'type': {'kind': 'STRUCT', 'name': '(anon)'}},
  {'name': '(anon)',
   'bits_offset': 0,
   'type': {'kind': 'STRUCT', 'name': '(anon)'}}]}

In [5]:
import sys

def print_as_list(name, s, num=10):
    print(f"{name} ({len(s)}): {list(s)[:num]}")


def diff_dict(old, new):
    added = {k: v for k, v in new.items() if k not in old}
    removed = {k: v for k, v in old.items() if k not in new}
    common = {k: (old[k], new[k]) for k in old.keys() if k in new}
    return added, removed, common


def _check_diff(d_old, d_new, kind, diff):
    f_old = d_old.filter_on_kind(kind)
    f_new = d_new.filter_on_kind(kind)

    print_as_list(f"Old {kind}", f_old.keys())
    print_as_list(f"New {kind}", f_new.keys())

    added, removed, common = diff_dict(f_old, f_new)

    print_as_list(f"Added {kind}", added)
    print_as_list(f"Removed {kind}", removed)
    print_as_list(f"Common {kind}", common)

    changed = {
        name: (old, new)
        for name, (old, new) in common.items()
        if old != new
    }
    print_as_list(f"Changed {kind}", changed.keys())

    for name, (old, new) in changed.items():
        reason = diff(old, new)
        lines = reason.strip().split("\n")
        print(f"{kind:12}{name}")
        for line in lines:
            print(f"{'':20} {line}")


class FileLogger:
    def __init__(self, name, print=True):
        self.print = print
        self.stdout = sys.stdout

        file_path = Path("output") / name
        file_path.parent.mkdir(parents=True, exist_ok=True)
        self.log = open(file_path, "w")

    def write(self, message):
        if self.print:
            self.stdout.write(message)
        self.log.write(message)

    def __enter__(self):
        sys.stdout = self
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout = self.stdout
        self.log.close()

def check_diff(d_old, d_new, kind, diff, print=False):
    assert d_old.path.parent == d_new.path.parent
    parent_name = d_old.path.parent.name
    name = f"{parent_name}/{d_old.short_name}->{d_new.short_name}.{kind.name}.log"
    with FileLogger(name, print=print):
        _check_diff(d_old, d_new, kind, diff)

In [6]:
def diff_struct(old, new):
    result = ""

    old_members = {m['name']: m for m in old['members']}
    new_members = {m['name']: m for m in new['members']}

    if '(anon)' in old_members or '(anon)' in new_members:
        return "Struct has anonymous fields"

    added, removed, common = diff_dict(old_members, new_members)

    # added field
    if added:
        result += f"Added fields:\n"
        for name, value in added.items():
            result += f"{name:>20}: {value['type']}\n"

    # removed field
    if removed:
        result += f"Removed fields:\n"
        for name, value in removed.items():
            result += f"{name:>20}: {value['type']}\n"

    # fields reordered
    if [n for n in old_members if n in common] != [n for n in new_members if n in common]:
        result += f"Fields reordered:\n"
        result += f"{'':>20} {list(old_members)}\n"
        result += f"{'':>20} {list(new_members)}\n"

    # fields changed type
    changed_types = {
        name: (old_value['type'], new_value['type'])
        for name, (old_value, new_value) in common.items()
        if old_value['type'] != new_value['type']
    }
    if changed_types:
        result += "Field type changed:\n"
        for name, (old_type, new_type) in changed_types.items():
            result += f"{name:>20}: {old_type}\n"
            result += f"{'':>20}->{new_type}\n"

    # fields changed offset
    old_offset = {name: old_members[name]
                  ['bits_offset'] for name in old_members}
    new_offset = {name: new_members[name]
                  ['bits_offset'] for name in new_members}
    layout_changed = old_offset != new_offset or old['size'] != new['size']
    if layout_changed and result == "":
        result += f"Layout changed\n"

    assert result, f"\n{old}\n{new}"
    return result


# check_diff(d1, d2, Kind.STRUCT, diff_struct, print=True)

In [7]:
# check_diff(d1, d2, Kind.UNION, diff_struct, print=True)

In [8]:
def diff_func(old, new):
    result = ""

    old_params = {p['name']: p for p in old['type']['params']}
    new_params = {p['name']: p for p in new['type']['params']}

    added, removed, common = diff_dict(old_params, new_params)

    # params added
    if added:
        result += f"Added params:\n"
        for name, value in added.items():
            result += f"{name:>20}: {value['type']}\n"

    # params removed
    if removed:
        result += f"Removed params:\n"
        for name, value in removed.items():
            result += f"{name:>20}: {value['type']}\n"

    # params reordered
    if [n for n in old_params if n in common] != [n for n in new_params if n in common]:
        result += f"Params reordered:\n"
        result += f"{'':>20} {list(old_params)}\n"
        result += f"{'':>20} {list(new_params)}\n"

    # params changed type
    changed_types = {
        name: (old_value['type'], new_value['type'])
        for name, (old_value, new_value) in common.items()
        if old_value['type'] != new_value['type']
    }
    if changed_types:
        result += "Param type changed:\n"
        for name, (old_type, new_type) in changed_types.items():
            result += f"{name:>20}: {old_type}\n"
            result += f"{'':>20}->{new_type}\n"

    # changed return value
    old_ret = old['type']['ret_type']
    new_ret = new['type']['ret_type']
    if old_ret != new_ret:
        result += f"Return type changed:\n"
        result += f"{'':>20}: {old_ret}\n"
        result += f"{'':>20}->{new_ret}\n"

    assert result, f"\n{old}\n{new}"
    return result


# check_diff(d1, d2, Kind.FUNC, diff_func, print=True)

In [9]:
def diff_enum(old, new):
    result = ""

    old_values = {v['name']: v for v in old['values']}
    new_values = {v['name']: v for v in new['values']}

    added, removed, common = diff_dict(old_values, new_values)

    # added value
    if added:
        result += f"Added values:\n"
        for name, value in added.items():
            result += f"{'':8}{name:40}: {value['val']}\n"

    # removed value
    if removed:
        result += f"Removed values:\n"
        for name, value in removed.items():
            result += f"{'':8}{name:40}: {value['val']}\n"

    # values changed
    changed_values = {
        name: (old_value['val'], new_value['val'])
        for name, (old_value, new_value) in common.items()
        if old_value['val'] != new_value['val']
    }
    if changed_values:
        result += "Value changed:\n"
        for name, (old_val, new_val) in changed_values.items():
            result += f"{'':8}{name:40}: {old_val} -> {new_val}\n"

    assert result, f"\n{old}\n{new}"

    return result


# check_diff(d1, d2, Kind.ENUM, diff_enum, print=True)

# Main

In [16]:
def get_json_paths(path):
    return sorted(
        (file for file in path.glob("*.json")),
        key=lambda name: tuple(map(int, name.stem.split("-")[0].split(".")))
    )


json_paths = get_json_paths(Path("data/16.04-x86"))

json_paths

[PosixPath('data/16.04-x86/4.4.0-210-generic.json'),
 PosixPath('data/16.04-x86/4.8.0-58-generic.json'),
 PosixPath('data/16.04-x86/4.10.0-42-generic.json'),
 PosixPath('data/16.04-x86/4.13.0-45-generic.json'),
 PosixPath('data/16.04-x86/4.15.0-142-generic.json')]

In [17]:
def diff_btf_file(path1, path2):
    d1 = BTF(path1)
    d2 = BTF(path2)
    print(f"Diffing {d1.path} and {d2.path}")
    check_diff(d1, d2, Kind.STRUCT, diff_struct)
    check_diff(d1, d2, Kind.UNION, diff_struct)
    check_diff(d1, d2, Kind.FUNC, diff_func)
    check_diff(d1, d2, Kind.ENUM, diff_enum)


def diff_all_btf_files(paths):
    for path1, path2 in list(zip(paths[:-1], paths[1:])) + [(paths[0], paths[-1])]:
        diff_btf_file(path1, path2)

In [13]:
diff_all_btf_files(json_paths)

Diffing data/16.04-x86/4.4.0-210-generic.json and data/16.04-x86/4.8.0-58-generic.json
Diffing data/16.04-x86/4.8.0-58-generic.json and data/16.04-x86/4.10.0-42-generic.json
Diffing data/16.04-x86/4.10.0-42-generic.json and data/16.04-x86/4.13.0-45-generic.json
Diffing data/16.04-x86/4.13.0-45-generic.json and data/16.04-x86/4.15.0-142-generic.json
Diffing data/16.04-x86/4.4.0-210-generic.json and data/16.04-x86/4.15.0-142-generic.json
